In [1]:
import pandas as pd
import empyrical

C:\Users\zhangyang01\AppData\Local\Continuum\anaconda2\envs\py36\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
df = pd.read_excel('D:/OneDrive/嘉实/资产配置/投资组合净值.xlsx', skiprows=1, index_col=0)

In [3]:
df.head()

,10/90组合（固收plus）,40/60组合（股债平衡）,10/90组合（固收plus）.1,40/60组合（股债平衡）.1,000906.SH,CBA00321.CS,10/90组合基准,40/60组合基准,10/90组合基准.1,40/60组合基准.1
2019-12-23,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000
2019-12-24,1.001108,1.003400,0.001108,0.003400,0.008594,0.000591,0.001391,0.003792,1.001391,1.003792
2019-12-25,1.001452,1.004127,0.000344,0.000725,0.000572,0.000478,0.000487,0.000515,1.001879,1.004310
2019-12-26,1.002273,1.007093,0.000820,0.002954,0.008359,0.000314,0.001118,0.003532,1.003000,1.007857
2019-12-27,1.002179,1.006679,-0.000094,-0.000411,-0.002576,0.000394,0.000097,-0.000794,1.003097,1.007056


In [4]:
empyrical.annual_volatility(df['10/90组合（固收plus）'].pct_change())

0.017008339055321085

In [6]:
empyrical.annual_volatility(df['40/60组合（股债平衡）'].pct_change())

0.07370779409523712